In [15]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
    
import pandas as pd
import numpy as np
import ast

import tasks.run_helper

In [16]:
df = pd.read_csv(
    "../data/100_annotators.csv",
    converters={"annot_personality_characteristics": ast.literal_eval},
)
df["toxicity"] = df.annotation.apply(lambda x: x[-1]).astype(int)
df["annot_politics"] = df.annot_personality_characteristics.apply(
    lambda x: x[0]
)
df.annot_age = pd.cut(df.annot_age, bins=4)
df.message_id = df.message_id.astype(str)
df["comment_key"] = df.conv_id + df.message_id

df = df.loc[
    :,
    [
        "conv_id",
        "message_id",
        "comment_key",
        "message",
        "toxicity",
        "annot_age",
        "annot_sex",
        "annot_sexual_orientation",
        "annot_demographic_group",
        "annot_current_employment",
        "annot_education_level",
        "annot_politics",
    ],
]
df = df.groupby(["conv_id", "message_id", "comment_key", "message"]).apply(
    lambda x: pd.Series(
        {
            col: x[col].tolist()
            for col in df.columns
            if col not in ["conv_id", "message_id", "comment_key", "message"]
        }
    ),
    include_groups=False,
)
df.head(1)

toxicity  \
conv_id                              message_id           comment_key                                        message                                                                                                 
250cc245-86c4-4e97-a8ce-31d5d6796f71 -1428289379724132680 250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379... well thats a no brainer hillary clinton gave hu...  [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...   

                                                                                                                                                                                                         annot_age  \
conv_id                              message_id           comment_key                                        message                                                                                                 
250cc245-86c4-4e97-a8ce-31d5d6796f71 -1428289379724132680 250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379... well thats a no brainer hillary clinton gave hu...  [(69.75, 88.0], (33.25, 51.5], (69.75, 88.0], ...   

                                                                                                                                                                                                         annot_sex  \
conv_id                              message_id           comment_key                                        message                                                                                                 
250cc245-86c4-4e97-a8ce-31d5d6796f71 -1428289379724132680 250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379... well thats a no brainer hillary clinton gave hu...  [female, non-binary, female, male, male, male,...   

                                                                                                                                                                                          annot_sexual_orientation  \
conv_id                              message_id           comment_key                                        message                                                                                                 
250cc245-86c4-4e97-a8ce-31d5d6796f71 -1428289379724132680 250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379... well thats a no brainer hillary clinton gave hu...  [homosexual, straight, straight, straight, bis...   

                                                                                                                                                                                           annot_demographic_group  \
conv_id                              message_id           comment_key                                        message                                                                                                 
250cc245-86c4-4e97-a8ce-31d5d6796f71 -1428289379724132680 250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379... well thats a no brainer hillary clinton gave hu...  [black, white, asian, other, asian, other, asi...   

                                                                                                                                                                                          annot_current_employment  \
conv_id                              message_id           comment_key                                        message                                                                                                 
250cc245-86c4-4e97-a8ce-31d5d6796f71 -1428289379724132680 250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379... well thats a no brainer hillary clinton gave hu...  [blue-collar, white-collar, unemployed, white-...   

                                                                                                                                                                                             annot_education_level  \
conv_id                              message_id           comment_key                                        message              

In [17]:
def results(
    df,
    discussion_id_col: str,
    sdb_column: str,
    value_col: str,
    comment_key_col: str,
) -> pd.DataFrame:
    res_ls = []

    # get results for each discussion
    discussion_ids = df.reset_index()[discussion_id_col].unique()
    for discussion_id in discussion_ids:
        discussion_df = df
        discussion_df = discussion_df.reset_index()
        discussion_df = discussion_df[
            discussion_df[discussion_id_col] == discussion_id
        ]

        res = tasks.run_helper.run_aposteriori(
            discussion_df,
            feature_col=sdb_column,
            value_col=value_col,
            comment_key_col=comment_key_col,
        )
        res_ls.append(res)

    return pd.concat(
        {_id: result for _id, result in zip(discussion_ids, res_ls)}, axis=1
    )

In [18]:
results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_age",
    value_col="toxicity",
    comment_key_col="comment_key",
)

250cc245-86c4-4e97-a8ce-31d5d6796f71  \
(14.927, 33.25]                              0.125992   
(33.25, 51.5]                                1.000000   
(51.5, 69.75]                                1.000000   
(69.75, 88.0]                                1.000000   

                 2feee653-15e9-4c38-b7a2-96fe26f50c9f  \
(14.927, 33.25]                              1.000000   
(33.25, 51.5]                                0.413794   
(51.5, 69.75]                                1.000000   
(69.75, 88.0]                                1.000000   

                 434ddba0-393a-424d-9c14-4837d454a221  \
(14.927, 33.25]                                   1.0   
(33.25, 51.5]                                     1.0   
(51.5, 69.75]                                     1.0   
(69.75, 88.0]                                     1.0   

                 a004f55d-03b1-4058-919a-6e6531aaf32f  
(14.927, 33.25]                                0.9294  
(33.25, 51.5]                                  1.0000  
(51.5, 69.75]                                  1.0000  
(69.75, 88.0]                                  1.0000

In [19]:
results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_sex",
    value_col="toxicity",
    comment_key_col="comment_key",
)

250cc245-86c4-4e97-a8ce-31d5d6796f71  \
female                                   1.00000   
male                                     1.00000   
non-binary                               0.24623   

            2feee653-15e9-4c38-b7a2-96fe26f50c9f  \
female                                       1.0   
male                                         1.0   
non-binary                                   1.0   

            434ddba0-393a-424d-9c14-4837d454a221  \
female                                  1.000000   
male                                    1.000000   
non-binary                              0.013026   

            a004f55d-03b1-4058-919a-6e6531aaf32f  
female                                  1.000000  
male                                    1.000000  
non-binary                              0.118275

In [20]:
results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_demographic_group",
    value_col="toxicity",
    comment_key_col="comment_key",
)

250cc245-86c4-4e97-a8ce-31d5d6796f71  \
asian                                   1.0   
black                                   1.0   
other                                   1.0   
white                                   1.0   

       2feee653-15e9-4c38-b7a2-96fe26f50c9f  \
asian                              1.000000   
black                              0.950841   
other                              1.000000   
white                              1.000000   

       434ddba0-393a-424d-9c14-4837d454a221  \
asian                              0.509981   
black                              1.000000   
other                              1.000000   
white                              1.000000   

       a004f55d-03b1-4058-919a-6e6531aaf32f  
asian                                   1.0  
black                                   1.0  
other                                   1.0  
white                                   1.0

In [21]:
results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_sexual_orientation",
    value_col="toxicity",
    comment_key_col="comment_key",
)

250cc245-86c4-4e97-a8ce-31d5d6796f71  \
bisexual                                     1.0   
homosexual                                   1.0   
other                                        1.0   
straight                                     1.0   

            2feee653-15e9-4c38-b7a2-96fe26f50c9f  \
bisexual                                     1.0   
homosexual                                   1.0   
other                                        1.0   
straight                                     1.0   

            434ddba0-393a-424d-9c14-4837d454a221  \
bisexual                                0.762771   
homosexual                              1.000000   
other                                   1.000000   
straight                                1.000000   

            a004f55d-03b1-4058-919a-6e6531aaf32f  
bisexual                                     1.0  
homosexual                                   1.0  
other                                        1.0  
straight                                     1.0

In [22]:
results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_current_employment",
    value_col="toxicity",
    comment_key_col="comment_key",
)

250cc245-86c4-4e97-a8ce-31d5d6796f71  \
blue-collar                                  0.782   
unemployed                                   1.000   
white-collar                                 1.000   

              2feee653-15e9-4c38-b7a2-96fe26f50c9f  \
blue-collar                                0.74942   
unemployed                                 1.00000   
white-collar                               1.00000   

              434ddba0-393a-424d-9c14-4837d454a221  \
blue-collar                                    1.0   
unemployed                                     1.0   
white-collar                                   1.0   

              a004f55d-03b1-4058-919a-6e6531aaf32f  
blue-collar                                    1.0  
unemployed                                     1.0  
white-collar                                   1.0

In [23]:
results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_education_level",
    value_col="toxicity",
    comment_key_col="comment_key",
)

250cc245-86c4-4e97-a8ce-31d5d6796f71  \
high-school                                   1.0   
none                                          1.0   
university                                    1.0   

             2feee653-15e9-4c38-b7a2-96fe26f50c9f  \
high-school                              0.936015   
none                                     1.000000   
university                               1.000000   

             434ddba0-393a-424d-9c14-4837d454a221  \
high-school                                   1.0   
none                                          1.0   
university                                    1.0   

             a004f55d-03b1-4058-919a-6e6531aaf32f  
high-school                                   1.0  
none                                          1.0  
university                                    1.0

In [25]:
results(
    df,
    discussion_id_col="conv_id",
    sdb_column="annot_politics",
    value_col="toxicity",
    comment_key_col="comment_key",
)

250cc245-86c4-4e97-a8ce-31d5d6796f71  \
apolitical                                           1.000000   
left-wing liberal                                    1.000000   
right-wing conservative                              0.464025   

                         2feee653-15e9-4c38-b7a2-96fe26f50c9f  \
apolitical                                           1.000000   
left-wing liberal                                    0.940768   
right-wing conservative                              1.000000   

                         434ddba0-393a-424d-9c14-4837d454a221  \
apolitical                                                1.0   
left-wing liberal                                         1.0   
right-wing conservative                                   1.0   

                         a004f55d-03b1-4058-919a-6e6531aaf32f  
apolitical                                           1.000000  
left-wing liberal                                    1.000000  
right-wing conservative                              0.726418